Copyright (c) Meta Platforms, Inc. and affiliates.
All rights reserved.

This source code is licensed under the license found in the
LICENSE file in the root directory of this source tree.

- First download the Nasa Exoplanets data from: https://www.kaggle.com/datasets/adityamishraml/nasaexoplanets
- Specifically you should have download the `cleaned_5250.csv` file which is what we load below.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_palette("husl")
import matplotlib as mpl
import torch

import os, sys
from parzen import ParzenWindowsGaussian
from utils import to_ndarray, parzen_plot, standardize


In [ ]:
df = pd.read_csv('cleaned_5250.csv')

In [ ]:
df

We will use the variable `radius_multiplier`, which is the radius with respect to the planet in the column `radius_wrt` (Jupiter or Earth). So we convert the ones as a function of Jupiter to the radius multiplier with respect to the Earth. For this, we multiply them by the constant 11.2089, which is the ratio between the radius of Jupiter and the Earth.

In [ ]:
df['radius_wrt_earth'] = ( (df['radius_wrt']=='Jupiter').astype(float)*df['radius_multiplier']*11.2089 + (df['radius_wrt']=='Earth').astype(float)*df['radius_multiplier'] )

In [ ]:
df

In [ ]:
df3 = pd.DataFrame()
df3['log_distance'] = np.log(df['distance'].values)
df3['log_stellar_magnitude'] = np.log(df['stellar_magnitude'].values)
df3['log_mass_multiplier'] = np.log(df['mass_multiplier'].values)
df3['log_radius_wrt_earth'] = np.log(df['radius_wrt_earth'].values)
df3['log_orbital_radius'] = np.log(df['orbital_radius'].values)
df3['log_orbital_period'] = np.log(df['orbital_period'].values)
df3['log_eccentricity'] = np.log(df['eccentricity'].values)

In [ ]:
df3

In [ ]:
x_y = df3[['log_stellar_magnitude', 'log_radius_wrt_earth']]
df_filter = x_y.isin([np.nan, np.inf, -np.inf])
# Mask df with the filter
x_y = x_y[~df_filter]
x_y.dropna(inplace=True)
x_y = x_y.values

x_y, _, _ = standardize(x_y)
x_y = torch.tensor(x_y)


In [ ]:
f, _, _ = parzen_plot(x_y, xlabel="Stellar magnitude", ylabel="Planet radius", levels=20, std=False, title="", arrows=False, scale=50, scale_units='height', sigma=0.15, n_points=100, cbar_label="Magnitude of density gradient", fontsize=16)

In [ ]:
f.savefig('exo.pdf')

In [ ]:
torch.corrcoef(x_y.T)[0,1]